In [18]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Reading the data
df = pd.read_csv('data.csv')
df.head()

# Preparing the train/test split
X = df.iloc[:,1:]
y = df.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)
print(X_train.shape)
print(y_train.shape)

(5114, 95)
(5114,)
Columns:  95


In [30]:
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
# TODO: in the future I would like to reduce these features to the highest
# correlated ones, remove outliers, scale up the non-bankruptcy data, and possibly
# expand out the data, but I ran out of time.
print("Columns: ", len(X_train.columns)) # Tons of columns to consider!

# Showing the drastic difference of data for bankruptcy vs no bankruptcy
sns.countplot(data=df, x='Bankrupt?')


# Giving the bankruptcy items more quantity to fix precision problem
# Abandoning this due to time limitations
# x = df.iloc[:,0].values.reshape(-1,1)
# y = df.iloc[:,1:].values
# over_sample = RandomOverSampler()
# X_ros, y_ros = over_sample.fit_sample(y,x)

ImportError: cannot import name 'delayed' from 'sklearn.utils.fixes' (c:\users\mvy12\pycharmprojects\uw_school\venv\lib\site-packages\sklearn\utils\fixes.py)

In [19]:
# Fitting the model and getting predictions
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline # Was going to use this with scaling the data and transforming, but abandonded
from sklearn.preprocessing import StandardScaler

# Could've spent a lot of time tinkering here, trying different models, but settling on this and a few settings that gave me data
parameters = {
    "max_depth": [8, 10, 12], # Making higher because of the amount of features / tinkering
    "min_samples_split": [4, 8],
    "criterion": ['entropy', 'gini'],
    "min_samples_leaf": [2]
}

# Building out the GridSearch for DTC to find the best params
grid_model = GridSearchCV(DecisionTreeClassifier(), parameters, cv=3)
grid_model.fit(X_train, y_train)
print("Best Params: ", grid_model.best_params_)

# Getting the predictions
y_pred = grid_model.predict(X_test)

In [20]:
# Analyzing some metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Accuracy is not valuable here as a metric, because the data is not scaled. Precision/recall are the more valuable metrics
# and in this case, both are quite low which shows this model/data is not very effective.
print("Accuracy: ", accuracy_score(y_true=y_test, y_pred=y_pred))
print("Precision: ", precision_score(y_true=y_test, y_pred=y_pred)) # would've wanted this to at least be above 50%
print("Recall: ", recall_score(y_true=y_test, y_pred=y_pred)) # ^^^, but I believe precision to be the more important metric here to accurately depict bankruptcies

Accuracy:  0.9695014662756598
Precision:  0.0
Recall:  0.0


c:\users\mvy12\pycharmprojects\uw_school\venv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Conceptual Overview
I wish I could've spent more time on this problem, as it was actually very interesting.
Time was not on my side this week. Data preparation would've really made this better (mentioned above),
as well as using some stacking & blending. Ultimately I was curious in this assignment how a
simple/naive approach to the data would work blindly.
Overall, this is a simple decision tree (random forest may've been better or bagging/etc). Which allows
me to see the interpretability of the graph. Decision trees are split each node based on an algorithm to even the weights
and are susceptible to over-fitting because with enough leaf nodes, every case may be accounted for.